In [2]:
import numpy as np
import pandas as pd

phenotypic_path = "D:\\AbideI\\phenotypic.csv"
dFC_path = "D:\\AbideI\\ProcessedData\\corr\\dFC.csv"

dFC = pd.read_csv(dFC_path)
phenotypic = pd.read_csv(phenotypic_path)

x = dFC.iloc[: , 1:]
y = phenotypic['DX_GROUP']

In [3]:
x.head()

,Precentral_L__Precentral_R_wk,Precentral_L__Precentral_R_st,Precentral_L__Frontal_Sup_L_wk,Precentral_L__Frontal_Sup_L_st,Precentral_L__Frontal_Sup_R_wk,Precentral_L__Frontal_Sup_R_st,Precentral_L__Frontal_Sup_Orb_L_wk,Precentral_L__Frontal_Sup_Orb_L_st,Precentral_L__Frontal_Sup_Orb_R_wk,Precentral_L__Frontal_Sup_Orb_R_st,...,Vermis_7__Vermis_9_wk,Vermis_7__Vermis_9_st,Vermis_7__Vermis_10_wk,Vermis_7__Vermis_10_st,Vermis_8__Vermis_9_wk,Vermis_8__Vermis_9_st,Vermis_8__Vermis_10_wk,Vermis_8__Vermis_10_st,Vermis_9__Vermis_10_wk,Vermis_9__Vermis_10_st
0,0.144,0.168,0.240,0.112,0.368,0.016,0.280,0.064,0.432,0.000,...,0.256,0.064,0.264,0.12,0.000,0.520,0.008,0.240,0.032,0.472
1,0.176,0.184,0.216,0.088,0.336,0.152,0.456,0.000,0.408,0.088,...,0.328,0.112,0.416,0.08,0.272,0.048,0.256,0.024,0.376,0.000
2,0.240,0.112,0.456,0.000,0.256,0.104,0.464,0.000,0.320,0.040,...,0.096,0.056,0.328,0.00,0.016,0.520,0.112,0.312,0.048,0.488
3,0.368,0.040,0.456,0.008,0.328,0.064,0.552,0.000,0.432,0.008,...,0.544,0.000,0.464,0.00,0.048,0.176,0.096,0.192,0.040,0.816
4,0.192,0.088,0.344,0.080,0.256,0.024,0.272,0.080,0.496,0.000,...,0.000,0.000,0.416,0.00,0.000,0.000,0.288,0.000,0.000,0.000


In [4]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: DX_GROUP, dtype: int64

In [5]:
from sklearn.feature_selection import GenericUnivariateSelect, SelectKBest, f_classif
# Step 1: 
selector = SelectKBest(f_classif, k = x.shape[1]//4)
X_selected = selector.fit_transform(x, y)

In [6]:
X_selected.shape

(884, 3335)

In [7]:
nFeatures = pd.read_csv("D:\\AbideI\\S4.csv")['feat_name']

In [8]:
nFeatures

0        Precentral_L__Rolandic_Oper_R_wk
1      Precentral_L__Supp_Motor_Area_L_st
2             Precentral_L__Fusiform_L_wk
3         Precentral_L__Parietal_Inf_L_wk
4        Precentral_L__SupraMarginal_R_st
                      ...                
835            Cerebelum_9_R__Vermis_9_wk
836     Cerebelum_10_L__Cerebelum_10_R_st
837          Cerebelum_10_R__Vermis_10_wk
838               Vermis_1_2__Vermis_3_st
839                 Vermis_7__Vermis_9_wk
Name: feat_name, Length: 840, dtype: object

In [9]:
step1 = pd.DataFrame(selector.get_feature_names_out())

In [10]:
step1

,0
0,Precentral_L__Precentral_R_wk
1,Precentral_L__Frontal_Sup_L_st
2,Precentral_L__Frontal_Sup_Orb_L_st
3,Precentral_L__Frontal_Mid_Orb_L_st
4,Precentral_L__Frontal_Mid_Orb_R_st
...,...
3330,Vermis_1_2__Vermis_3_st
3331,Vermis_1_2__Vermis_9_st
3332,Vermis_4_5__Vermis_8_st
3333,Vermis_6__Vermis_10_st


In [11]:
missing = []
for i in range (len(nFeatures)):
    if((nFeatures[i] in step1.values) == False):
        print(nFeatures[i] + ' not included in STEP1')
        missing.append(nFeatures[i])

In [12]:
len(missing)

0

In [13]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.svm import LinearSVC
import time 
import os

In [14]:
lsvm_estimator = LinearSVC(dual = 'auto', max_iter=5000)
n_features_to_select = 840 
cv = StratifiedKFold(5, shuffle=True, random_state=123)

rfecv = RFECV(estimator=lsvm_estimator, step=2, cv=cv, scoring='balanced_accuracy', min_features_to_select=n_features_to_select)
t = time.time()
X_rfecv = rfecv.fit_transform(X_selected, y)

elapsed = time.time() - t
print("Time = " + str(elapsed/60) + " minutes")

Time = 118.6860087354978 minutes


In [15]:
X_rfecv.shape

(884, 2123)

In [16]:
feature_ranking_cv = rfecv.ranking_
top_n_features_cv = np.argsort(feature_ranking_cv)[:n_features_to_select]

selected_features_cv = X_selected[:, top_n_features_cv]

selected_features_cv.shape

(884, 840)

In [18]:
param_grid = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10, 100, 300]}
linear_svm_classifier = LinearSVC(dual='auto', max_iter= 8000)
cv = StratifiedKFold(5, shuffle=True, random_state=123)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(selected_features_cv, y)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {selected_features_cv.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=0.1, dual='auto', max_iter=8000)
Best Hyperparameters: {'C': 0.1}
Selected Features: 840 features
Mean Cross-Validated Score: 0.7345362941568012 for Hyperparameters: {'C': 0.01}
Mean Cross-Validated Score: 0.7744766656365394 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.7623745324807049 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.759056631246137 for Hyperparameters: {'C': 1.2}
Mean Cross-Validated Score: 0.760276143441259 for Hyperparameters: {'C': 1.3}
Mean Cross-Validated Score: 0.7625373223030476 for Hyperparameters: {'C': 1.4}
Mean Cross-Validated Score: 0.7637568344981694 for Hyperparameters: {'C': 1.5}
Mean Cross-Validated Score: 0.7651432273094662 for Hyperparameters: {'C': 2}
Mean Cross-Validated Score: 0.7641207071427439 for Hyperparameters: {'C': 3}
Mean Cross-Validated Score: 0.7639278059081761 for Hyperparameters: {'C': 4}
Mean Cross-Validated Score: 0.7639278059081761 for Hyperparameters: {'C': 5}
Mean 

In [18]:
lsvm_estimator = LinearSVC(dual = 'auto', max_iter=5000)
n_features_to_select = 840 
cv = StratifiedKFold(5, shuffle=True, random_state=123)

rfecv = RFECV(estimator=lsvm_estimator, step=2, cv=cv, scoring='balanced_accuracy', min_features_to_select=n_features_to_select)
t = time.time()
X_rfecv = rfecv.fit_transform(step1_data, y)

elapsed = time.time() - t
print("Time = " + str(elapsed/60) + " minutes")

Time = 118.22904812097549 minutes


In [19]:
X_rfecv.shape

(884, 2123)

In [20]:
mask = rfecv.get_support()
selected_features = selected_columns[mask]

In [21]:
selected_features

array(['Precentral_L__Precentral_R_wk', 'Precentral_L__Frontal_Sup_L_st',
       'Precentral_L__Frontal_Sup_Orb_L_st', ...,
       'Vermis_1_2__Vermis_9_st', 'Vermis_6__Vermis_10_st',
       'Vermis_7__Vermis_9_wk'], dtype=object)

In [24]:
missing = []
selected_features_df = pd.DataFrame(selected_features)
for i in range (len(nFeatures)):
    if((nFeatures[i] in selected_features_df.values) == False):
        print(nFeatures[i] + ' not included')
        missing.append(nFeatures[i])
        
len(missing)

0

In [25]:
step2_data = x[selected_features].copy()
step2_data

,Precentral_L__Precentral_R_wk,Precentral_L__Frontal_Sup_L_st,Precentral_L__Frontal_Sup_Orb_L_st,Precentral_L__Frontal_Inf_Tri_L_wk,Precentral_L__Rolandic_Oper_R_wk,Precentral_L__Supp_Motor_Area_L_st,Precentral_L__Supp_Motor_Area_R_st,Precentral_L__Cingulum_Post_R_wk,Precentral_L__Cingulum_Post_R_st,Precentral_L__Cuneus_L_st,...,Cerebelum_9_R__Vermis_9_st,Cerebelum_10_L__Cerebelum_10_R_st,Cerebelum_10_L__Vermis_1_2_st,Cerebelum_10_L__Vermis_4_5_st,Cerebelum_10_R__Vermis_10_wk,Vermis_1_2__Vermis_3_wk,Vermis_1_2__Vermis_3_st,Vermis_1_2__Vermis_9_st,Vermis_6__Vermis_10_st,Vermis_7__Vermis_9_wk
0,0.144000,0.112000,0.064000,0.208000,0.408000,0.104000,0.032000,0.264000,0.056000,0.096000,...,0.792000,0.032000,0.040000,0.168000,0.192000,0.472000,0.016000,0.008000,0.184000,0.256000
1,0.176000,0.088000,0.000000,0.424000,0.344000,0.000000,0.000000,0.544000,0.000000,0.048000,...,0.112000,0.032000,0.000000,0.032000,0.472000,0.416000,0.000000,0.000000,0.016000,0.328000
2,0.240000,0.000000,0.000000,0.352000,0.248000,0.032000,0.032000,0.208000,0.136000,0.000000,...,0.992000,0.000000,0.000000,0.000000,0.000000,0.448000,0.000000,0.160000,0.000000,0.096000
3,0.368000,0.008000,0.000000,0.400000,0.376000,0.168000,0.096000,0.296000,0.016000,0.104000,...,0.000000,0.000000,0.000000,0.000000,0.416000,0.144000,0.312000,0.336000,0.024000,0.544000
4,0.192000,0.080000,0.080000,0.200000,0.488000,0.080000,0.088000,0.200000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.416000,0.296000,0.096000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,0.074286,0.000000,0.028571,0.234286,0.434286,0.040000,0.034286,0.377143,0.000000,0.000000,...,0.097143,0.000000,0.017143,0.000000,0.337143,0.211429,0.125714,0.000000,0.000000,0.497143
880,0.171429,0.040000,0.011429,0.371429,0.274286,0.028571,0.005714,0.297143,0.097143,0.034286,...,0.022857,0.022857,0.011429,0.028571,0.331429,0.205714,0.057143,0.011429,0.068571,0.485714
881,0.331429,0.045714,0.085714,0.388571,0.262857,0.074286,0.028571,0.302857,0.000000,0.011429,...,0.154286,0.028571,0.051429,0.022857,0.348571,0.417143,0.022857,0.074286,0.034286,0.411429
882,0.177143,0.000000,0.000000,0.440000,0.354286,0.068571,0.034286,0.240000,0.040000,0.034286,...,0.171429,0.085714,0.000000,0.034286,0.348571,0.394286,0.028571,0.000000,0.000000,0.422857


lSVM Classifier

In [26]:
param_grid = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10, 100, 300]}
linear_svm_classifier = LinearSVC(dual='auto', max_iter= 5000)
cv = StratifiedKFold(5, shuffle=True, random_state=123)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(step2_data, y)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {step2_data.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=100, dual='auto', max_iter=5000)
Best Hyperparameters: {'C': 100}
Selected Features: 2123 features
Mean Cross-Validated Score: 0.8189290539469722 for Hyperparameters: {'C': 0.01}
Mean Cross-Validated Score: 0.8839325999619645 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.9028114748252746 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.9028114748252746 for Hyperparameters: {'C': 1.2}
Mean Cross-Validated Score: 0.9028114748252746 for Hyperparameters: {'C': 1.3}
Mean Cross-Validated Score: 0.9028114748252746 for Hyperparameters: {'C': 1.4}
Mean Cross-Validated Score: 0.9028114748252746 for Hyperparameters: {'C': 1.5}
Mean Cross-Validated Score: 0.9038641064042219 for Hyperparameters: {'C': 2}
Mean Cross-Validated Score: 0.9026295385029874 for Hyperparameters: {'C': 3}
Mean Cross-Validated Score: 0.9049016822770568 for Hyperparameters: {'C': 4}
Mean Cross-Validated Score: 0.9049016822770568 for Hyperparameters: {'C': 5}
Me

In [48]:
param_grid = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10, 100, 300]}
linear_svm_classifier = LinearSVC(dual='auto', max_iter= 5000)
#cv = StratifiedKFold(5, shuffle=True, random_state=123)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=5, scoring='balanced_accuracy')
grid_search.fit(step2_data, y)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {step2_data.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=3, dual='auto', max_iter=5000)
Best Hyperparameters: {'C': 3}
Selected Features: 2123 features
Mean Cross-Validated Score: 0.8164318075563799 for Hyperparameters: {'C': 0.01}
Mean Cross-Validated Score: 0.8769383231113013 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.8872505011965324 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.8872505011965324 for Hyperparameters: {'C': 1.2}
Mean Cross-Validated Score: 0.8872505011965324 for Hyperparameters: {'C': 1.3}
Mean Cross-Validated Score: 0.8872505011965324 for Hyperparameters: {'C': 1.4}
Mean Cross-Validated Score: 0.8884700133916544 for Hyperparameters: {'C': 1.5}
Mean Cross-Validated Score: 0.8895226449706017 for Hyperparameters: {'C': 2}
Mean Cross-Validated Score: 0.8919616693608455 for Hyperparameters: {'C': 3}
Mean Cross-Validated Score: 0.8919616693608455 for Hyperparameters: {'C': 4}
Mean Cross-Validated Score: 0.8909090377818982 for Hyperparameters: {'C': 5}
Mean C

SELECT TOP 840 Features

In [42]:
rfecv.ranking_

array([  1,   1,   1, ..., 213,   1,   1])

In [39]:
feature_rank = np.column_stack((selector.get_feature_names_out(), rfecv.ranking_))
feature_rank

array([['Precentral_L__Precentral_R_wk', 1],
       ['Precentral_L__Frontal_Sup_L_st', 1],
       ['Precentral_L__Frontal_Sup_Orb_L_st', 1],
       ...,
       ['Vermis_4_5__Vermis_8_st', 213],
       ['Vermis_6__Vermis_10_st', 1],
       ['Vermis_7__Vermis_9_wk', 1]], dtype=object)

In [40]:
sorted_features = feature_rank[feature_rank[:, 1].argsort()]
sorted_features

array([['Precentral_L__Precentral_R_wk', 1],
       ['ParaHippocampal_R__Cerebelum_4_5_L_st', 1],
       ['ParaHippocampal_R__Cerebelum_4_5_R_wk', 1],
       ...,
       ['Cingulum_Ant_L__Vermis_6_st', 606],
       ['Precuneus_R__Heschl_R_wk', 607],
       ['Temporal_Pole_Mid_R__Cerebelum_4_5_R_wk', 607]], dtype=object)

In [41]:
selected_features = sorted_features[:840, 0]
selected_features

array(['Precentral_L__Precentral_R_wk',
       'ParaHippocampal_R__Cerebelum_4_5_L_st',
       'ParaHippocampal_R__Cerebelum_4_5_R_wk',
       'ParaHippocampal_R__Cerebelum_4_5_R_st',
       'ParaHippocampal_R__Cerebelum_6_R_wk',
       'ParaHippocampal_R__Cerebelum_7b_L_st',
       'ParaHippocampal_R__Cerebelum_7b_R_st',
       'ParaHippocampal_R__Cerebelum_8_R_st',
       'ParaHippocampal_R__Cerebelum_3_L_wk',
       'ParaHippocampal_R__Vermis_3_st', 'ParaHippocampal_R__Vermis_6_wk',
       'ParaHippocampal_R__Vermis_7_st', 'Amygdala_L__Amygdala_R_wk',
       'Amygdala_L__Amygdala_R_st', 'Amygdala_L__Calcarine_L_wk',
       'Amygdala_L__Calcarine_R_wk', 'Amygdala_L__Occipital_Sup_L_st',
       'ParaHippocampal_R__Vermis_4_5_wk',
       'Amygdala_L__Occipital_Mid_L_st',
       'ParaHippocampal_R__Cerebelum_Crus2_R_st',
       'ParaHippocampal_R__Temporal_Inf_R_wk',
       'ParaHippocampal_L__Cerebelum_6_L_st',
       'ParaHippocampal_L__Cerebelum_8_L_st',
       'ParaHippocampal_L__Ce

In [45]:
final_data = x[selected_features]

In [47]:
param_grid = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10, 100, 300]}
linear_svm_classifier = LinearSVC(dual='auto', max_iter= 8000)
cv = StratifiedKFold(5, shuffle=True, random_state=123)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(final_data, y)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {final_data.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=0.1, dual='auto', max_iter=8000)
Best Hyperparameters: {'C': 0.1}
Selected Features: 840 features
Mean Cross-Validated Score: 0.7367974730185898 for Hyperparameters: {'C': 0.01}
Mean Cross-Validated Score: 0.7744766656365394 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.7679755424016228 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.7658552235376155 for Hyperparameters: {'C': 1.2}
Mean Cross-Validated Score: 0.7648135568709489 for Hyperparameters: {'C': 1.3}
Mean Cross-Validated Score: 0.7637609252920015 for Hyperparameters: {'C': 1.4}
Mean Cross-Validated Score: 0.7616666270463874 for Hyperparameters: {'C': 1.5}
Mean Cross-Validated Score: 0.7639387708204568 for Hyperparameters: {'C': 2}
Mean Cross-Validated Score: 0.7627082937130542 for Hyperparameters: {'C': 3}
Mean Cross-Validated Score: 0.7637609252920015 for Hyperparameters: {'C': 4}
Mean Cross-Validated Score: 0.7637609252920015 for Hyperparameters: {'C': 5}
Mea

# Split Data to testing and training

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(step2_data, y, test_size=0.1, random_state=42)

grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {X_train.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=0.9, dual='auto', max_iter=5000)
Best Hyperparameters: {'C': 0.9}
Selected Features: 2123 features
Mean Cross-Validated Score: 0.8716190476190476 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.8779999999999999 for Hyperparameters: {'C': 0.2}
Mean Cross-Validated Score: 0.8782857142857143 for Hyperparameters: {'C': 0.3}
Mean Cross-Validated Score: 0.8793333333333333 for Hyperparameters: {'C': 0.4}
Mean Cross-Validated Score: 0.8781428571428572 for Hyperparameters: {'C': 0.5}
Mean Cross-Validated Score: 0.8781428571428572 for Hyperparameters: {'C': 0.6}
Mean Cross-Validated Score: 0.8794761904761905 for Hyperparameters: {'C': 0.7}
Mean Cross-Validated Score: 0.8808095238095237 for Hyperparameters: {'C': 0.8}
Mean Cross-Validated Score: 0.8821428571428571 for Hyperparameters: {'C': 0.9}
Mean Cross-Validated Score: 0.880952380952381 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.8785714285714287 for Hyperparameters: {'C': 1

In [57]:
accuracy = best_linear_svm_classifier.score(X_test, y_test)
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9213483146067416


# LOGO

In [58]:
def extract_site(entry):
    site = entry.split('\\')[-1].split('_')[0]
    return site

In [59]:
sites = pd.read_csv(dFC_path).iloc[: , 0].apply(extract_site)
sites

0      Caltech
1      Caltech
2      Caltech
3      Caltech
4      Caltech
        ...   
879       Yale
880       Yale
881       Yale
882       Yale
883       Yale
Name: Unnamed: 0, Length: 884, dtype: object

In [60]:
X = step2_data
if len(X) == len(sites):
    sites_reshaped = sites.values.reshape(-1, 1)
    X_with_site = np.concatenate((X, sites_reshaped), axis=1)

groups = sites.drop_duplicates()

In [63]:
from sklearn.model_selection import LeaveOneGroupOut

logo = LeaveOneGroupOut()

In [66]:
linear_svm_classifier = LinearSVC(dual='auto', max_iter= 8000)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=logo, scoring='balanced_accuracy')
grid_search.fit(X_with_site[:, :-1], y, groups=sites)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=10, dual='auto', max_iter=8000)
Best Hyperparameters: {'C': 10}
Mean Cross-Validated Score: 0.8815772738092016 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.9030090176896751 for Hyperparameters: {'C': 0.2}
Mean Cross-Validated Score: 0.9042752850661908 for Hyperparameters: {'C': 0.3}
Mean Cross-Validated Score: 0.9058541774927268 for Hyperparameters: {'C': 0.4}
Mean Cross-Validated Score: 0.9089848415230447 for Hyperparameters: {'C': 0.5}
Mean Cross-Validated Score: 0.912797477688622 for Hyperparameters: {'C': 0.6}
Mean Cross-Validated Score: 0.9136625295917362 for Hyperparameters: {'C': 0.7}
Mean Cross-Validated Score: 0.910595527389786 for Hyperparameters: {'C': 0.8}
Mean Cross-Validated Score: 0.9122295143178905 for Hyperparameters: {'C': 0.9}
Mean Cross-Validated Score: 0.9124988528225231 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.9141913456764422 for Hyperparameters: {'C': 10}
Mean Cross-Validated Score: 0.914

In [67]:
logo.get_n_splits(X_with_site, y, groups)

17

In [70]:
from sklearn.metrics import accuracy_score

l_svc = grid_search.best_estimator_
test_accuracy = []
train_accuracy = []
counter = 0
for train_index, test_index in logo.split(X_with_site, y, sites):
    counter += 1
    X_train, X_test = X_with_site[train_index, :-1], X_with_site[test_index, :-1]
    y_train, y_test = y[train_index], y[test_index]
    print("Training model: ", counter)
    l_svc.fit(X_train, y_train)
    y_train_pred = l_svc.predict(X_train)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_accuracy.append(train_acc)
    
    test_acc = l_svc.score(X_test, y_test)
    test_accuracy.append(test_acc)
    
    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

print("Mean Train Accuracy:", np.mean(train_accuracy))
print("Mean Test Accuracy:", np.mean(test_accuracy))

Training model:  1
Train Accuracy: 1.0000, Test Accuracy: 1.0000
Training model:  2
Train Accuracy: 1.0000, Test Accuracy: 0.9189
Training model:  3
Train Accuracy: 1.0000, Test Accuracy: 0.8462
Training model:  4
Train Accuracy: 1.0000, Test Accuracy: 0.8525
Training model:  5
Train Accuracy: 1.0000, Test Accuracy: 0.9286
Training model:  6
Train Accuracy: 1.0000, Test Accuracy: 0.8830
Training model:  7
Train Accuracy: 1.0000, Test Accuracy: 0.9130
Training model:  8
Train Accuracy: 1.0000, Test Accuracy: 0.9200
Training model:  9
Train Accuracy: 1.0000, Test Accuracy: 0.8444
Training model:  10
Train Accuracy: 1.0000, Test Accuracy: 0.9231
Training model:  11
Train Accuracy: 1.0000, Test Accuracy: 0.9697
Training model:  12
Train Accuracy: 1.0000, Test Accuracy: 0.9167
Training model:  13
Train Accuracy: 1.0000, Test Accuracy: 1.0000
Training model:  14
Train Accuracy: 1.0000, Test Accuracy: 0.9200
Training model:  15
Train Accuracy: 1.0000, Test Accuracy: 0.8850
Training model:  16


Find best C first

In [19]:
lsvc_est = LinearSVC(dual = 'auto', max_iter=5000)
param_grid = {'C': [0.01, 0.1, 1, 1.2, 1.3, 1.4, 1.5, 2, 3, 4,  5, 10, 100, 300]}

grid_search = GridSearchCV(estimator=lsvc_est, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_selected, y)
best_c = grid_search.best_params_['C']

In [24]:
best_c

1

In [25]:
lsvm_estimator = LinearSVC(dual = 'auto', max_iter=8000, C = best_c)
n_features_to_select = 840 
rfecv_1 = RFECV(estimator=lsvm_estimator, step=2, cv=5, scoring='balanced_accuracy')
t = time.time()
X_rfecv_1 = rfecv_1.fit_transform(X_selected, y)

elapsed = time.time() - t
print("Time = " + str(elapsed/60) + " minutes")


Time = 127.95523413419724 minutes


In [26]:
feature_ranking_cv1 = rfecv_1.ranking_
top_n_features_cv1 = np.argsort(feature_ranking_cv1)[:n_features_to_select]

selected_features_cv1 = X_selected[:, top_n_features_cv]

selected_features_cv1.shape

(884, 840)

In [27]:
grid_search = GridSearchCV(lsvm_estimator, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(selected_features_cv1, y)

best_lsvm_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {selected_features_cv1.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=0.1, dual='auto', max_iter=8000)
Best Hyperparameters: {'C': 0.1}
Selected Features: 840 features
Mean Cross-Validated Score: 0.7345362941568012 for Hyperparameters: {'C': 0.01}
Mean Cross-Validated Score: 0.7744766656365394 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.7623745324807049 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.759056631246137 for Hyperparameters: {'C': 1.2}
Mean Cross-Validated Score: 0.760276143441259 for Hyperparameters: {'C': 1.3}
Mean Cross-Validated Score: 0.7625373223030476 for Hyperparameters: {'C': 1.4}
Mean Cross-Validated Score: 0.7637568344981694 for Hyperparameters: {'C': 1.5}
Mean Cross-Validated Score: 0.7651432273094662 for Hyperparameters: {'C': 2}
Mean Cross-Validated Score: 0.7641207071427439 for Hyperparameters: {'C': 3}
Mean Cross-Validated Score: 0.7639278059081761 for Hyperparameters: {'C': 4}
Mean Cross-Validated Score: 0.7639278059081761 for Hyperparameters: {'C': 5}
Mean 

# Using features from S4

In [49]:
df = pd.DataFrame(columns=nFeatures)

In [50]:
df

feat_name,Precentral_L__Rolandic_Oper_R_wk,Precentral_L__Supp_Motor_Area_L_st,Precentral_L__Fusiform_L_wk,Precentral_L__Parietal_Inf_L_wk,Precentral_L__SupraMarginal_R_st,Precentral_L__Precuneus_R_wk,Precentral_L__Temporal_Sup_L_wk,Precentral_L__Temporal_Pole_Sup_R_wk,Precentral_L__Cerebelum_Crus2_R_wk,Precentral_R__Frontal_Inf_Tri_L_wk,...,Cerebelum_8_R__Vermis_8_st,Cerebelum_8_R__Vermis_9_st,Cerebelum_9_L__Vermis_4_5_wk,Cerebelum_9_L__Vermis_9_st,Cerebelum_9_L__Vermis_10_wk,Cerebelum_9_R__Vermis_9_wk,Cerebelum_10_L__Cerebelum_10_R_st,Cerebelum_10_R__Vermis_10_wk,Vermis_1_2__Vermis_3_st,Vermis_7__Vermis_9_wk


In [51]:
selected_columns = df.columns

In [53]:
data = x[selected_columns].copy()

In [54]:
data.head()

,Precentral_L__Rolandic_Oper_R_wk,Precentral_L__Supp_Motor_Area_L_st,Precentral_L__Fusiform_L_wk,Precentral_L__Parietal_Inf_L_wk,Precentral_L__SupraMarginal_R_st,Precentral_L__Precuneus_R_wk,Precentral_L__Temporal_Sup_L_wk,Precentral_L__Temporal_Pole_Sup_R_wk,Precentral_L__Cerebelum_Crus2_R_wk,Precentral_R__Frontal_Inf_Tri_L_wk,...,Cerebelum_8_R__Vermis_8_st,Cerebelum_8_R__Vermis_9_st,Cerebelum_9_L__Vermis_4_5_wk,Cerebelum_9_L__Vermis_9_st,Cerebelum_9_L__Vermis_10_wk,Cerebelum_9_R__Vermis_9_wk,Cerebelum_10_L__Cerebelum_10_R_st,Cerebelum_10_R__Vermis_10_wk,Vermis_1_2__Vermis_3_st,Vermis_7__Vermis_9_wk
0,0.408,0.104,0.352,0.184,0.040,0.448,0.368,0.472,0.448,0.320,...,0.448,0.448,0.360,0.128,0.376,0.0,0.032,0.192,0.016,0.256
1,0.344,0.000,0.392,0.224,0.168,0.464,0.376,0.448,0.368,0.368,...,0.160,0.144,0.392,0.104,0.248,0.2,0.032,0.472,0.000,0.328
2,0.248,0.032,0.488,0.208,0.128,0.328,0.376,0.344,0.560,0.192,...,0.672,0.712,0.520,0.000,0.472,0.0,0.000,0.000,0.000,0.096
3,0.376,0.168,0.272,0.288,0.008,0.296,0.448,0.256,0.456,0.184,...,0.608,0.120,0.256,0.488,0.032,0.0,0.000,0.416,0.312,0.544
4,0.488,0.080,0.504,0.056,0.000,0.312,0.528,0.440,0.392,0.248,...,0.000,0.000,0.344,0.000,0.488,0.0,0.000,0.416,0.096,0.000


In [55]:
param_grid = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 10, 100]}
cv = StratifiedKFold(5, shuffle=True, random_state=123)
grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(data, y)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=10, dual='auto', max_iter=5000)
Best Hyperparameters: {'C': 10}
Mean Cross-Validated Score: 0.9611143401638694 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.9703889423762659 for Hyperparameters: {'C': 0.2}
Mean Cross-Validated Score: 0.9751151662466917 for Hyperparameters: {'C': 0.3}
Mean Cross-Validated Score: 0.9812578943406394 for Hyperparameters: {'C': 0.4}
Mean Cross-Validated Score: 0.9824924622418738 for Hyperparameters: {'C': 0.5}
Mean Cross-Validated Score: 0.9837270301431085 for Hyperparameters: {'C': 0.6}
Mean Cross-Validated Score: 0.9837270301431085 for Hyperparameters: {'C': 0.7}
Mean Cross-Validated Score: 0.9847686968097751 for Hyperparameters: {'C': 0.8}
Mean Cross-Validated Score: 0.9860032647110095 for Hyperparameters: {'C': 0.9}
Mean Cross-Validated Score: 0.9860032647110095 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.988275408485079 for Hyperparameters: {'C': 10}
Mean Cross-Validated Score: 0.98

Split Data to testing and training

In [72]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=42)

grid_search = GridSearchCV(linear_svm_classifier, param_grid, cv=cv, scoring='balanced_accuracy')
grid_search.fit(X_train, y_train)

best_linear_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_


print(f"Best LinearSVM Classifier: {best_linear_svm_classifier}")
print(f"Best Hyperparameters: {best_params}")
print(f"Selected Features: {X_train.shape[1]} features")

cv_results = grid_search.cv_results_
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean Cross-Validated Score: {mean_score} for Hyperparameters: {params}")

Best LinearSVM Classifier: LinearSVC(C=10, dual='auto', max_iter=8000)
Best Hyperparameters: {'C': 10}
Selected Features: 840 features
Mean Cross-Validated Score: 0.9691428571428572 for Hyperparameters: {'C': 0.1}
Mean Cross-Validated Score: 0.9741904761904762 for Hyperparameters: {'C': 0.2}
Mean Cross-Validated Score: 0.9780476190476189 for Hyperparameters: {'C': 0.3}
Mean Cross-Validated Score: 0.981904761904762 for Hyperparameters: {'C': 0.4}
Mean Cross-Validated Score: 0.9859047619047618 for Hyperparameters: {'C': 0.5}
Mean Cross-Validated Score: 0.9870952380952381 for Hyperparameters: {'C': 0.6}
Mean Cross-Validated Score: 0.9870952380952381 for Hyperparameters: {'C': 0.7}
Mean Cross-Validated Score: 0.9870952380952381 for Hyperparameters: {'C': 0.8}
Mean Cross-Validated Score: 0.9870952380952381 for Hyperparameters: {'C': 0.9}
Mean Cross-Validated Score: 0.9870952380952381 for Hyperparameters: {'C': 1}
Mean Cross-Validated Score: 0.9884285714285713 for Hyperparameters: {'C': 10}


In [73]:
accuracy = best_linear_svm_classifier.score(X_test, y_test)
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9887640449438202
